In [1]:
from google.colab import drive
import pandas as pd
import gzip
import re
from tqdm.auto import tqdm

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
rewiew_path = './drive/My Drive/nlp_hw/Software.json.gz' 
metadata_path = './drive/My Drive/nlp_hw/meta_Software.json.gz' 

In [3]:
true = True
false = False

In [4]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [5]:
review_df = getDF(rewiew_path)
metadata_df = getDF(metadata_path)

In [6]:
review_df.head(2)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,4.0,True,"03 11, 2014",A240ORQ2LF9LUI,0077613252,{'Format:': ' Loose Leaf'},Michelle W,The materials arrived early and were in excell...,Material Great,1394496000,NaN,NaN
1,4.0,True,"02 23, 2014",A1YCCU0YRLS0FE,0077613252,{'Format:': ' Loose Leaf'},Rosalind White Ames,I am really enjoying this book with the worksh...,Health,1393113600,NaN,NaN


In [7]:
metadata_df.head(2)

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,[],,[],,HOLT PHYSICS LESSON PRESENTATION CD-ROM QUICK ...,[],,HOLT. RINEHART AND WINSTON,[],"25,550 in Software (",[],Software,,</div>,.a-box-inner{background-color:#fff}#alohaBuyBo...,0030672120,[],[],NaN
1,[],,"[, <b>Latin rhythms that will get your kids si...",,"Sing, Watch, &amp; Learn Spanish (DVD + Guide)...",[],,McGraw Hill,[],"15,792 in Software (",[],Software,,</div>,,0071480935,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN


# 1. 3 способа найти упоминания товаров в отзывах


1. Найти все n-граммы из заголовков (убрав стоп-слова), попытаться найти самые частотные из них, а их же в свою очередь искать в отзывах (+: не найдется ничего лишнего; -: в отзывах могут появляться слова, не встречающиеся в названии продукта, но тем не менее являющиеся его обозначением, их упустим)

2. С помощью tf-idf найти наиболее значимые слова в отзывах, затем отсортировать их по частям речи (+: найдем специфичные слова (которые, например, могут входить в название модели); -: многое упустим)

3. Найти ключевые слова и их совпадение с заголовками (+: не упустит длинные названия и в целом биграммы; -: неточные и скорее всего неполные данные)

# 2. Первый способ

Я выбрала первый способ. На данном этапе выделим те слова в отзывах, которые входят в название, на следующем найдем соседей, которые тоже входят в заголовок. 

In [ ]:
review_df = review_df[['asin', 'reviewText']]
df = pd.merge(review_df.groupby('asin').agg(list), metadata_df[['asin', 'title']], on='asin')
df = df.dropna()

In [38]:
def clean_text(text):
  text = re.sub('[^a-z0-9 ]', ' ', text.lower())
  text = re.sub(' +', ' ', text)
  return text.strip(' ')

In [41]:
df['title'] = df['title'].apply(lambda x: clean_text(x))
df['reviewText'] = df['reviewText'].apply(lambda x: ' # '.join([clean_text(i) for i in x]))

In [42]:
df

,asin,reviewText,title
0,0030672120,good resource,holt physics lesson presentation cd rom quick ...
1,0071480935,i expected better quality the video is fuzzy a...,sing watch amp learn spanish dvd guide 16 fun ...
2,007329506X,the ad for this does not say that this only al...,connect with learnsmart access card for microb...
3,0073513458,the required access code for my course is for ...,learnsmart standalone access card for prescott...
4,0073525758,returned,anatomy amp physiology revealed student access...
...,...,...,...
26782,B01HEFZJC2,the images are deceptive the pendant is so sma...,925 sterling silver angel wings heart pendant ...
26783,B01HEFZKEE,is very nice but it was much smaller than i ex...,925 sterling silver love heart opal pendant ne...
26784,B01HF3G4BS,it s a joke # i have multiple licenses of the ...,intego mac internet security x9 1 mac 1 year p...
26785,B01HF41TKI,no instructions no help unless you want to pay...,versacheck x9 small and medium business 2016 3...


In [44]:
import nltk
from nltk.corpus import stopwords
import spacy

#nltk.download('stopwords')
stoplist = stopwords.words('english')
sp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [46]:
asins = list(df['asin'])

In [87]:
asin_words = {a:[] for a in asins}

for asin in tqdm(asins, total=len(asins)):
  rev = df[df['asin']==asin]['reviewText'].values[0]
  tit = df[df['asin']==asin]['title'].values[0]
  words = set(rev.split(' '))
  for w in words:
    if w not in stoplist and w in tit and w:
      pos_tag = sp(w)[0].pos_
      if pos_tag == 'NOUN' or pos_tag == 'PROPN':
        asin_words[asin].append(w)


  0%|          | 0/26787 [00:00<?, ?it/s]

In [89]:
df_ = pd.DataFrame({'asin': asin_words.keys(), 'NE': asin_words.values()})
df_NE = pd.merge(df, df_, on='asin')

In [92]:
df_NE

,asin,reviewText,title,NE
0,0030672120,good resource,holt physics lesson presentation cd rom quick ...,[]
1,0071480935,i expected better quality the video is fuzzy a...,sing watch amp learn spanish dvd guide 16 fun ...,"[dvd, music, fun, video]"
2,007329506X,the ad for this does not say that this only al...,connect with learnsmart access card for microb...,"[access, card]"
3,0073513458,the required access code for my course is for ...,learnsmart standalone access card for prescott...,"[access, prescott, microbiology]"
4,0073525758,returned,anatomy amp physiology revealed student access...,[]
...,...,...,...,...
26782,B01HEFZJC2,the images are deceptive the pendant is so sma...,925 sterling silver angel wings heart pendant ...,[]
26783,B01HEFZKEE,is very nice but it was much smaller than i ex...,925 sterling silver love heart opal pendant ne...,[opal]
26784,B01HF3G4BS,it s a joke # i have multiple licenses of the ...,intego mac internet security x9 1 mac 1 year p...,[]
26785,B01HF41TKI,no instructions no help unless you want to pay...,versacheck x9 small and medium business 2016 3...,[]


In [91]:
# df_NE.to_csv('./drive/My Drive/nlp_hw/df_NE.csv', sep='\t', encoding='utf-8')

# 3. Находим биграммы

In [121]:
import nltk
from nltk.collocations import *
from nltk.tokenize import RegexpTokenizer

In [119]:
df_['NE'] = df_NE['NE'].apply(lambda x: ' '.join(x))
NEs = list(set(' '.join(list(df_['NE'])).split(' ')))
del NEs[NEs.index('')]

In [124]:
tokenizer = RegexpTokenizer(r'\w+')
tok_texts = df_NE['reviewText'].apply(tokenizer.tokenize).to_list()

In [168]:
finder = BigramCollocationFinder.from_documents(tok_texts)

# 4. Сортируем биграммы 

(one - хотя бы одно слово входит в NEs, double - входят оба слова, как планировалось в пункте 2)


In [177]:
bigram_measures = nltk.collocations.BigramAssocMeasures()

In [202]:
def find_bigrams(results, n):
  correct = []
  double = []
  for bg in results:
    if len(correct) < n:
      if bg[0][0] in NEs or bg[0][1] in NEs:
        correct.append(bg)
    if len(double) < n:
      if bg[0][0] in NEs and bg[0][1] in NEs:
        double.append(bg)
    if len(double) == n and len(correct) == n:
      break
  return correct, double

In [ ]:
jaccard_one, jaccard_double = find_bigrams(finder.score_ngrams(bigram_measures.jaccard), 15)

In [182]:
jaccard_one

[(('agatha', 'christies'), 1.0),
 (('aoki', 'lapis'), 1.0),
 (('blackmar', 'diemer'), 1.0),
 (('calderas', 'eserver'), 1.0),
 (('cobuild', 'engl'), 1.0),
 (('cuerpo', 'humano'), 1.0),
 (('enciclo', 'pedia'), 1.0),
 (('eserver', 'distrubtion'), 1.0),
 (('irus', 'productas'), 1.0),
 (('jacquie', 'lawson'), 1.0),
 (('mwd', '1470'), 1.0),
 (('pathagragum', 'numerology'), 1.0),
 (('pcwa', 'de80'), 1.0),
 (('prin', 'tshop'), 1.0),
 (('sheri', 'spaine'), 1.0)]

In [183]:
jaccard_double

[(('blackmar', 'diemer'), 1.0),
 (('jacquie', 'lawson'), 1.0),
 (('pcwa', 'de80'), 1.0),
 (('tohoku', 'zunko'), 1.0),
 (('palo', 'alto'), 0.8783783783783784),
 (('feng', 'shui'), 0.8235294117647058),
 (('notre', 'dame'), 0.8125),
 (('rolie', 'polie'), 0.75),
 (('mavis', 'beacon'), 0.741421568627451),
 (('thomson', 'reuters'), 0.7058823529411765),
 (('sarzo', 'workingman'), 0.6666666666666666),
 (('suze', 'orman'), 0.6666666666666666),
 (('yuki', 'kaai'), 0.6666666666666666),
 (('viet', 'nam'), 0.625),
 (('wi', 'fi'), 0.6174496644295302)]

In [184]:
lr_one, lr_double = find_bigrams(finder.score_ngrams(bigram_measures.likelihood_ratio), 15)

In [185]:
lr_one

[(('this', 'product'), 282066.45702038996),
 (('turbo', 'tax'), 248146.89649027036),
 (('a', 'lot'), 243695.96285201042),
 (('h', 'r'), 207154.89235820822),
 (('windows', '7'), 196014.3209526453),
 (('my', 'computer'), 168545.01487303048),
 (('this', 'software'), 162445.71250098463),
 (('this', 'program'), 154382.25950154493),
 (('r', 'block'), 153622.03622146617),
 (('lot', 'of'), 147860.42468512597),
 (('the', 'program'), 146552.4015086915),
 (('tech', 'support'), 141189.4209628146),
 (('customer', 'service'), 140621.1885661913),
 (('windows', '8'), 128676.1243527483),
 (('user', 'friendly'), 122984.6633744475)]

In [186]:
lr_double

[(('turbo', 'tax'), 248146.89649027036),
 (('customer', 'service'), 140621.1885661913),
 (('internet', 'security'), 84130.74977850678),
 (('credit', 'card'), 59712.69219544165),
 (('trend', 'micro'), 58524.659750762956),
 (('web', 'site'), 54729.24839564377),
 (('data', 'hook'), 50809.182812423125),
 (('microsoft', 'office'), 42099.677265910694),
 (('hook', 'product'), 38519.20078347141),
 (('error', 'message'), 37509.3260540527),
 (('ms', 'office'), 36439.714688618886),
 (('family', 'tree'), 33430.499627406425),
 (('serial', 'number'), 29365.053494384134),
 (('product', 'key'), 26650.98015125568),
 (('clip', 'art'), 23088.41723641813)]

In [187]:
pmi_one, pmi_double = find_bigrams(finder.score_ngrams(bigram_measures.pmi), 15)

In [188]:
pmi_one

[(('agatha', 'christies'), 25.300591427591407),
 (('blackmar', 'diemer'), 25.300591427591407),
 (('cobuild', 'engl'), 25.300591427591407),
 (('cuerpo', 'humano'), 25.300591427591407),
 (('irus', 'productas'), 25.300591427591407),
 (('mwd', '1470'), 25.300591427591407),
 (('pathagragum', 'numerology'), 25.300591427591407),
 (('prin', 'tshop'), 25.300591427591407),
 (('sheri', 'spaine'), 25.300591427591407),
 (('tohoku', 'zunko'), 25.300591427591407),
 (('2900r', 'transceiver'), 24.300591427591407),
 (('abdellah', 'jaber'), 24.300591427591407),
 (('biblia', 'hebraica'), 24.300591427591407),
 (('calderas', 'eserver'), 24.300591427591407),
 (('dauntless', 'atpl'), 24.300591427591407)]

In [189]:
pmi_double

[(('blackmar', 'diemer'), 25.300591427591407),
 (('tohoku', 'zunko'), 25.300591427591407),
 (('hayabusa', 'gsxr'), 24.300591427591407),
 (('jacquie', 'lawson'), 24.300591427591407),
 (('karsten', 'muller'), 24.300591427591407),
 (('nekomura', 'iroha'), 24.300591427591407),
 (('confucius', 'buddha'), 23.715628926870252),
 (('leviton', 'vizia'), 23.715628926870252),
 (('madrigal', 'velasco'), 23.715628926870252),
 (('sarzo', 'workingman'), 23.715628926870252),
 (('yuki', 'kaai'), 23.715628926870252),
 (('lacy', 'snowflakes'), 23.300591427591407),
 (('otherworld', 'omens'), 23.300591427591407),
 (('pxw', 'fs7'), 23.300591427591407),
 (('suzuki', 'hayabusa'), 23.300591427591407)]

На мой взгляд, pmi ранжирует выше наиболее подходящие коллокации, jaccard чуть хуже и likelihood_ratio хуже всех.

# 5. Примеры для 5 товаров


In [209]:
def find_bigrams(results):
  correct = []
  for bg in results:
    if bg[0][0] in NEs or bg[0][1] in NEs:
      correct.append(bg)
  return correct

In [210]:
pmi_one = find_bigrams(finder.score_ngrams(bigram_measures.pmi))

In [215]:
import random 

sample = {word:[] for word in random.sample(NEs, 5)}

for bg in tqdm(pmi_one, total=len(pmi_one)):
  for word in bg[0]:
    if word in sample and len(sample[word]) < 5:
      sample[word].append(bg)


  0%|          | 0/1156432 [00:00<?, ?it/s]

In [217]:
for w in sample:
  if sample[w]:
    print(f'{w}\n---')
    for bg in sample[w]:
      print(' '.join(bg[0]))
  print('\n\n')

intelliscore
---
intelliscore finale
called intelliscore
bought intelliscore
intelliscore which
intelliscore when



hygiene
---
dental hygiene
hygiene boards
hygiene our
hygiene game
about hygiene



laura
---
laura heinrichs
precedetors laura
laura hawkins
laura unbeknownst
laura vincent



processor
---
1055t processor
1091t processor
15mhz processor
1600x processor
25hz processor



photosuit
---
recommend photosuit
photosuit it



